In [1]:
import time
import itertools
from KeyRecoveryScheme import h, Cipher, p
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
from leak import leak
import base64

def lagrange_polynomial(i, x_lst, gf): # The polynomial L_i(x) according to the paper
    R.<X> = gf['X']
    result = 1
    for j in range(len(x_lst)):
        if (j == i):
            continue
        X_term = (X - x_lst[j]) / (x_lst[i] - x_lst[j])
        result *= X_term
    return result

def get_pts(lock, possible_answers, m, n, p): # enumerates possibilities at each x coordinate, based on possible values of share at each coordinate
    s_i = [int(s_ij) for s_ij in lock.decode().split(",")]
    pts = []
    assert(len(s_i) == n)
    for x in range(0, n):
        assert(len(possible_answers[x]) == m)
        s_ij = s_i[x]
        cand = []
        for i in range(m):
            cip = Cipher((x + 1, possible_answers[x][i]), p)
            dec = cip.decrypt(s_ij)
            cand.append(dec)
        pts.append((x + 1, cand))
    return pts

def precompute_y_times_L(pts, m, n, gf):
    (x_lst, y_cand) = zip(*pts)
    for cand in y_cand:
        assert(len(cand) == m)
    assert(len(x_lst) == len(y_cand))
    assert(n == len(x_lst))
    y_times_L = []
    for i in range(n):
        y_times_L_i = []
        for j in range(m):
            y_times_L_i.append(y_cand[i][j] * lagrange_polynomial(i, x_lst, gf))
        y_times_L.append(y_times_L_i)
    return y_times_L

def check_pair(p1, p2, k, gf): # check if pair of polynomials solve the problem
    R.<X> = gf['X']
    poly_diff = p1 - p2
    return (poly_diff.degree(X) <= k)

def poly_key(poly, k, n):
    poly_coeff = poly.coefficients(sparse=True)
    poly_coeff.extend([0] * (n - len(poly_coeff)))
    return poly_coeff[k + 1:]
    

def key_recovery_attack(lock, possible_answers, k, m, n, p):
    begin_time = time.time()
    pts = get_pts(lock, possible_answers, m, n, p)
    get_pts_time = time.time()
    print(f"Generated set of candidate y values for each x value ({get_pts_time - begin_time} seconds)")
    gf = GF(p)
    y_times_L = precompute_y_times_L(pts, m, n, gf)
    precompute_time = time.time()
    print(f"Precomputed lookup table for y_ij * L_i ({precompute_time - get_pts_time} seconds)")
    middle = len(y_times_L) // 2
    y_times_L_left = y_times_L[:middle]
    y_times_L_right = y_times_L[middle:]
    UV_merge = [(sum(poly), "U") for poly in itertools.product(*y_times_L_left)]
    left_half_time = time.time()
    print(f"Possible left halves of interpolated polynomial sum computed ({left_half_time - precompute_time} seconds)")
    for poly in itertools.product(*y_times_L_right):
        UV_merge.append((-1 * sum(poly), "V"))
    right_half_time = time.time()
    print(f"Possible negative right halves of interpolated polynomial sum computed ({right_half_time - left_half_time} seconds)")
    UV_merge.sort(key=lambda x: poly_key(x[0], k, n))
    merge_time = time.time()
    print(f"Sorted left and right halves ({merge_time - right_half_time} seconds)")
    for i in range(len(UV_merge) - 1):
        if (check_pair(UV_merge[i][0], UV_merge[i + 1][0], k, gf)):
            if (UV_merge[i][1] == "U" and UV_merge[i + 1][1] == "V"):
                poly = UV_merge[i][0] - UV_merge[i + 1][0]
            elif (UV_merge[i][1] == "V" and UV_merge[i + 1][1] == "U"):
                poly = UV_merge[i + 1][0] - UV_merge[i][0]
            else:
                continue
            match_time = time.time()
            print(f"Match found ({match_time - merge_time} seconds)")
            key = h(poly.coefficients(sparse=False)[0])
            print(f"key = {str(key)}")
            end_time = time.time()
            print(f"Total time of full attack: {end_time - begin_time} seconds")
            return key
    return None

In [2]:
possible_answers = leak
possible_answers

[['thomas',
  'miku',
  'harris',
  'baltimore',
  'hipple',
  'ayer',
  'ashe',
  'arming',
  'acre',
  'beckinham'],
 ['russell',
  'tom',
  'jerry',
  'ashley',
  'johnny',
  'benjamin',
  'fred',
  'gerry',
  'dumbo',
  'sesame'],
 ['san diego',
  'san jose',
  'sacramento',
  'san francisco',
  'new york city',
  'los angeles',
  'san bernandino',
  'trenton',
  'detroit',
  'columbus'],
 ['birdemic',
  'fateful findings',
  'the last airbender',
  'troll 2',
  'batman and robin',
  'fantastic four',
  'disaster movie',
  'manos',
  'catwoman',
  'secrets of dumbledore'],
 ['decision to leave',
  'oldboy',
  'memories of murder',
  'blade runner 2049',
  'puss in boots 2',
  'into the spider-verse',
  'blade runner',
  'mother',
  'parasite',
  'portrait of a lady on fire'],
 ['barcarolle',
  'american boy',
  'gimme gimme gimme',
  'naatu naatu',
  'smells like teen spirit',
  'something in the way',
  'party in the usa',
  'thrift shop',
  'not afraid',
  'levels'],
 ['call me m

In [3]:
lock = b'190629033538511228468646206251559516519,238294487125633318381847448413858389148,27340971959422876078804955756782104475,201405021234643450057313886115967464005,73047844731805600891321835883882766999,4080439589675999097316970863193093516,171336484628641692271874903491662195159,215992970631530033943277468343898207908,75760716032030419139156362841569837182,101564264340921992580619866260582515217,81830384793133738498165413702028233091,39443721770710429698377379354299974512'
key = key_recovery_attack(lock=lock, possible_answers=possible_answers, k=10 - 1, m=10, n=12, p=p)

Generated set of candidate y values for each x value (0.013661861419677734 seconds)
Precomputed lookup table for y_ij * L_i (0.04422450065612793 seconds)
Possible left halves of interpolated polynomial sum computed (13.38466191291809 seconds)
Possible negative right halves of interpolated polynomial sum computed (23.348328828811646 seconds)
Sorted left and right halves (88.66975378990173 seconds)
Match found (17.86655282974243 seconds)
key = b'\x9a\xd4\xa5^\x89I#\x03VC\x8d=R\xdbG\xbb'
Total time of full attack: 145.9511411190033 seconds


In [4]:
flag_aes = base64.b64decode("Vdouw1720Iy1cbXulXWnIhEFkjmmQSUstXAkR8yP4HbigN8Gu8toysBRlVEqeWbv")
aes_cipher = AES.new(key, AES.MODE_ECB)
unpad(aes_cipher.decrypt(flag_aes), 16)

b'SDCTF{62d6f17b98a5b23d6b3be676197ef56d}'